In [20]:
import pandas as pd
import numpy as np
import ast

In [21]:
data = pd.read_csv("FINAL_RE_PLC_review_tokenized_okt_30377.csv")

In [22]:
data['review_tokens'] = data['review_tokens'].apply(lambda x: ast.literal_eval(x))

In [23]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.preprocessing import LabelEncoder

In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['review_tokens'])
X = tokenizer.texts_to_sequences(data['review_tokens']) # Sequence 변환
max_len = 82
X = pad_sequences(X, max_len)
le = LabelEncoder()
y = le.fit_transform(data.plc)
y = to_categorical(y)

In [25]:
vocab_size = len(tokenizer.word_index) + 2

In [26]:
embedding_matrix = np.load('embedding_mat.npy')

## Attention + Bidirectional LSTM 구현

In [27]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)
    
    def call(self, values, query):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        
        score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))
        
        attention_weights = tf.nn.softmax(score, axis = 1)
        
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis = 1)
        
        return context_vector, attention_weights

In [28]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, values, query): 
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

In [32]:
sequence_input = Input(shape = (max_len, ), dtype = 'int32')
embedded_sequences = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False, 
                               mask_zero = True)(sequence_input)
lstm = Bidirectional(LSTM(units = 128, dropout = 0.2, return_sequences = True))(embedded_sequences)
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(units = 129, dropout = 0.2, return_sequences=True, return_state=True))(lstm)
    
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
    
attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)
        
dense = Dense(units = 300, activation="relu")(context_vector)
dropout = Dropout(rate = 0.2)(dense)
output = Dense(349, activation="softmax")(dropout)
    
model = Model(inputs=sequence_input, outputs=output)
    
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tfa.metrics.FBetaScore(num_classes=349, average='micro', beta=0.5)])

In [33]:
early_stopping = EarlyStopping(monitor='loss', patience=5)
val_f1_score = float('-inf')
model_checkpoint = ModelCheckpoint('Attention_BiLSTM.h5', monitor = 'fbeta_score', mode = 'max', verbose = 1, save_best_only = True)

In [34]:
history = model.fit(X, y, epochs=150, callbacks=[early_stopping], batch_size=128)

Epoch 1/150
238/238 [==============================] - 184s 730ms/step - loss: 4.9444 - fbeta_score: 0.0607
Epoch 2/150
238/238 [==============================] - 174s 732ms/step - loss: 4.1954 - fbeta_score: 0.1382
Epoch 3/150
238/238 [==============================] - 175s 734ms/step - loss: 3.6096 - fbeta_score: 0.2328
Epoch 4/150
238/238 [==============================] - 174s 732ms/step - loss: 3.2921 - fbeta_score: 0.2866
Epoch 5/150
238/238 [==============================] - 175s 734ms/step - loss: 3.0493 - fbeta_score: 0.3287
Epoch 6/150
238/238 [==============================] - 175s 735ms/step - loss: 2.9070 - fbeta_score: 0.3518
Epoch 7/150
238/238 [==============================] - 174s 733ms/step - loss: 2.7612 - fbeta_score: 0.3793
Epoch 8/150
238/238 [==============================] - 176s 738ms/step - loss: 2.6216 - fbeta_score: 0.4010
Epoch 9/150
238/238 [==============================] - 174s 732ms/step - loss: 2.5471 - fbeta_score: 0.4109
Epoch 10/150
238/238 [======

In [90]:
model.save_weights('attention_weights.h5')

In [97]:
def create_model():
    
    class BahdanauAttention(tf.keras.Model):
        def __init__(self, units):
            super(BahdanauAttention, self).__init__()
            self.W1 = Dense(units)
            self.W2 = Dense(units)
            self.V = Dense(1)

        def call(self, values, query): 
            hidden_with_time_axis = tf.expand_dims(query, 1)
            score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
            attention_weights = tf.nn.softmax(score, axis=1)
            context_vector = attention_weights * values
            context_vector = tf.reduce_sum(context_vector, axis=1)
            return context_vector, attention_weights
    
    
    sequence_input = Input(shape = (max_len, ), dtype = 'int32')
    embedded_sequences = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False, 
                               mask_zero = True)(sequence_input)
    lstm = Bidirectional(LSTM(units = 128, dropout = 0.2, return_sequences = True))(embedded_sequences)
    lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(units = 129, dropout = 0.2, return_sequences=True, return_state=True))(lstm)
    
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])

    attention = BahdanauAttention(64) # 가중치 크기 정의
    context_vector, attention_weights = attention(lstm, state_h)

    dense = Dense(units = 300, activation="relu")(context_vector)
    dropout = Dropout(rate = 0.2)(dense)
    output = Dense(349, activation="softmax")(dropout)

    model2 = Model(inputs=sequence_input, outputs=output)

    model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tfa.metrics.FBetaScore(num_classes=349, average='micro', beta=0.5)])
    
    return model2

In [98]:
model2 = create_model()

In [104]:
model2.load_weights('./attention_weights.h5')

In [105]:
model2.evaluate(X, y)

950/950 [==============================] - 114s 115ms/step - loss: 0.1143 - fbeta_score: 0.9565


[0.16182972490787506, 0.9300128221511841]

In [106]:
model.evaluate(X, y)

950/950 [==============================] - 118s 115ms/step - loss: 0.1618 - fbeta_score: 0.9300


[0.16182973980903625, 0.9300128221511841]